# 1. 국토교통부 실거래가 공개시스템 데이터 분석
- 대상 데이터 - 아파트(매매) 실거래가 2025년 12월 1달 기준
- 본 분석에서는 ㎡당 가격 기준으로 지역별 주택 가격 수준을 비교하기 위해 아파트 매매 실거래가 데이터를 우선적으로 활용
- 고려사항
    - **실제 청년층의 주거 이동은 아파트 매매에 국한되지 않으며, 전·월세 거래, 연립·다세대, 오피스텔, 단독·다가구 주택 등 다양한 주택 유형과 거래 형태가 혼재되어 있어, 이를 고려하여 이후 분석에서는 거래 유형 및 주택 유형을 확장할 필요가 있음.**

In [ ]:
import pandas as pd

df = pd.read_csv(
    "./data/아파트(매매)_실거래가_202512_20260128.csv",
    encoding="cp949",
    sep=",",
    skiprows=15,
    engine="python",
)

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# 거래금액 전처리 str -> int
df["거래금액(만원)"] = (
    df["거래금액(만원)"].str.replace(",", "", regex=False).astype(int)
)

In [ ]:
# 컬럼 신규 생성
df["㎡당가격"] = df["거래금액(만원)"] / df["전용면적(㎡)"]

In [ ]:
# “청년 이동” 분석인데 84㎡, 120㎡ 섞이면 논리 무너짐
df_young = df[(df["전용면적(㎡)"] >= 40) & (df["전용면적(㎡)"] <= 60)].copy()

In [ ]:
# 시군구까지만
df_young["시군구_정제"] = df_young["시군구"].str.split().str[:3].str.join(" ")

In [ ]:
# 지역별 주거비 대표값 생성
region_price = (
    df_young.groupby("시군구_정제")
    .agg(
        평균거래금액=("거래금액(만원)", "mean"),
        평균_면적당가격=("㎡당가격", "mean"),
        거래건수=("NO", "count"),
    )
    .reset_index()
)

region_price = region_price.rename(
    columns={
        "avg_price": "평균거래금액",
        "avg_price_per_m2": "평균_㎡당가격",
        "cnt": "거래건수",
    }
)

In [ ]:
df.head()

In [ ]:
# 지역별 평균 ㎡당 가격 분포
region_price["평균_면적당가격"].describe()

In [ ]:
# 거래건수 필터링
region_price_f = region_price[region_price["거래건수"] >= 30]

In [ ]:
# ㎡당 가격 기준 상위 10
cheap_top10 = region_price_f.sort_values("평균_면적당가격").head(10)
cheap_top10

In [ ]:
# ㎡당 가격 기준 하위 10
expensive_top10 = region_price_f.sort_values("평균_면적당가격", ascending=False).head(
    10
)
expensive_top10

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# 정렬 (낮은 값 → 위, 높은 값 → 아래)
plot_df = expensive_top10.sort_values("평균_면적당가격")

plt.figure(figsize=(8, 6))
plt.barh(plot_df["시군구_정제"], plot_df["평균_면적당가격"])

plt.title("㎡당 가격 상위 10 지역", fontsize=13, fontweight="bold")
plt.xlabel("㎡당 평균 가격")
plt.ylabel("")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# 정렬 (낮은 값 → 위, 높은 값 → 아래)
plot_df = cheap_top10.sort_values("평균_면적당가격")

plt.figure(figsize=(8, 6))
plt.barh(plot_df["시군구_정제"], plot_df["평균_면적당가격"])

plt.title("㎡당 가격 하위 10 지역", fontsize=13, fontweight="bold")
plt.xlabel("㎡당 평균 가격")
plt.ylabel("")

plt.tight_layout()
plt.show()